<a href="https://colab.research.google.com/github/AhmedSSoliman/Leveraging-Pretrained-Language-Models-for-Code-Generation/blob/master/LUKE_Marian_CoNaLa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
lafrom google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
import torch
torch.cuda.empty_cache()

# **Install dependencies**

In [ ]:
%%capture
!pip install tensorflow
!pip install keras
!pip install netron
!pip install nltk

!pip install git+https://github.com/huggingface/transformers
!pip install datasets
!pip install nlp
!pip install rouge-score
#!pip install sacrebleu
!pip install git-python
!pip install sentencepiece
#!pip install torchdata
#!pip install transformers
!pip install sacrebleu==1.5.1
!pip install tree-sitter
!pip install evaluate

# **Import Libraries**

In [ ]:
import transformers
import datasets
from datasets import load_dataset, load_metric
import logging
from transformers import BertTokenizer, GPT2Tokenizer, GPT2TokenizerFast, EncoderDecoderModel, Trainer, TrainingArguments, BertTokenizerFast
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

logging.basicConfig(level=logging.INFO)

import types
import argparse
import logging
from functools import partial
import json

import torch
from torch import optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import (
    BertGenerationConfig,
    BertGenerationEncoder,
    BertTokenizer,
    EncoderDecoderModel,
    EncoderDecoderConfig,
    GPT2LMHeadModel,
    GPT2TokenizerFast,
    Trainer,
    TrainingArguments,
    get_cosine_schedule_with_warmup,
)

import sacrebleu
import random
import numpy as np
#from tensorboardX import SummaryWriter


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
%cd /content/drive/MyDrive/models/CodeGeneration/New/CoNaLa/LUKE-Marian

/content/drive/MyDrive/models/CodeGeneration/New/CoNaLa/LUKE-Marian


# **Load Data**

In [ ]:
from datasets import load_dataset

raw_dataset = load_dataset("AhmedSSoliman/CoNaLa-Large")

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/AhmedSSoliman___csv/AhmedSSoliman--CoNaLa-Large-87e6f6b36beed736/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
train_dataset= raw_dataset["train"]
val_dataset = raw_dataset["validation"]
test_dataset = raw_dataset["test"]

In [ ]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['intent', 'snippet'],
        num_rows: 24687
    })
    test: Dataset({
        features: ['intent', 'snippet'],
        num_rows: 500
    })
    validation: Dataset({
        features: ['intent', 'snippet'],
        num_rows: 1237
    })
})

# **Tokenizer**

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("studio-ousia/luke-base", use_fast = True)
tokenizer

Downloading:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/836 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/15.3M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/33.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

PreTrainedTokenizer(name_or_path='studio-ousia/luke-base', vocab_size=50265, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True), 'additional_special_tokens': [AddedToken("<ent>", rstrip=False, lstrip=False, single_word=False, normalized=True), AddedToken("<ent2>", rstrip=Fa

# **Model**

In [ ]:
model = EncoderDecoderModel.from_encoder_decoder_pretrained("studio-ousia/luke-base", "Helsinki-NLP/opus-mt-en-nl")
model

Downloading:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

Some weights of the model checkpoint at studio-ousia/luke-base were not used when initializing LukeModel: ['lm_head.dense.weight', 'entity_predictions.transform.LayerNorm.weight', 'lm_head.bias', 'entity_predictions.transform.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'entity_predictions.transform.LayerNorm.bias', 'entity_predictions.transform.dense.weight', 'entity_predictions.bias']
- This IS expected if you are initializing LukeModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LukeModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/316M [00:00<?, ?B/s]

Some weights of the model checkpoint at Helsinki-NLP/opus-mt-en-nl were not used when initializing MarianForCausalLM: ['model.encoder.layers.1.final_layer_norm.bias', 'model.encoder.layers.1.fc2.weight', 'model.encoder.layers.5.final_layer_norm.weight', 'model.encoder.layers.2.fc2.bias', 'model.encoder.layers.2.final_layer_norm.bias', 'model.encoder.layers.3.self_attn_layer_norm.bias', 'model.encoder.layers.2.self_attn.v_proj.bias', 'model.encoder.layers.5.self_attn.v_proj.weight', 'model.encoder.layers.0.fc2.bias', 'model.encoder.layers.2.self_attn.out_proj.bias', 'model.encoder.layers.2.fc1.weight', 'model.encoder.layers.3.self_attn.q_proj.bias', 'model.encoder.layers.0.self_attn.q_proj.weight', 'model.encoder.layers.2.fc2.weight', 'model.encoder.layers.1.self_attn.v_proj.bias', 'model.encoder.layers.1.self_attn.q_proj.bias', 'model.encoder.layers.4.final_layer_norm.weight', 'model.encoder.layers.5.self_attn_layer_norm.weight', 'model.encoder.layers.4.fc1.bias', 'final_logits_bias', 

EncoderDecoderModel(
  (encoder): LukeModel(
    (embeddings): LukeEmbeddings(
      (word_embeddings): Embedding(50267, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (entity_embeddings): LukeEntityEmbeddings(
      (entity_embeddings): Embedding(500000, 256, padding_idx=0)
      (entity_embedding_dense): Linear(in_features=256, out_features=768, bias=False)
      (position_embeddings): Embedding(514, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): LukeEncoder(
      (layer): ModuleList(
        (0): LukeLayer(
          (attention): LukeAttention(
            (self): LukeSelfAttention(
              (query): Linear(in_features=

In [ ]:
model.config

EncoderDecoderConfig {
  "_commit_hash": null,
  "decoder": {
    "_name_or_path": "Helsinki-NLP/opus-mt-en-nl",
    "_num_labels": 3,
    "activation_dropout": 0.0,
    "activation_function": "swish",
    "add_bias_logits": false,
    "add_cross_attention": true,
    "add_final_layer_norm": false,
    "architectures": [
      "MarianMTModel"
    ],
    "attention_dropout": 0.0,
    "bad_words_ids": [
      [
        67027
      ]
    ],
    "bos_token_id": 0,
    "chunk_size_feed_forward": 0,
    "classif_dropout": 0.0,
    "classifier_dropout": 0.0,
    "cross_attention_hidden_size": null,
    "d_model": 512,
    "decoder_attention_heads": 8,
    "decoder_ffn_dim": 2048,
    "decoder_layerdrop": 0.0,
    "decoder_layers": 6,
    "decoder_start_token_id": 67027,
    "decoder_vocab_size": 67028,
    "diversity_penalty": 0.0,
    "do_sample": false,
    "dropout": 0.1,
    "early_stopping": false,
    "encoder_attention_heads": 8,
    "encoder_ffn_dim": 2048,
    "encoder_layerdrop": 0.

In [ ]:
model.base_model.config.encoder.num_hidden_layers = 6

In [ ]:
model.tie_weights = True

In [ ]:
model.base_model.config.decoder_start_token_id = model.base_model.config.decoder.bos_token_id

In [ ]:
model.base_model.config.bos_token_id = tokenizer.bos_token_id
model.base_model.config.eos_token_id = tokenizer.eos_token_id
model.base_model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
model.config.max_length =512

In [ ]:
model.save_pretrained("LUKE-Marian-base-model")

In [ ]:
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("./LUKE-Marian-base-model")
#model=AutoModelForSeq2SeqLM.from_pretrained("./LUKE-marian-training")
model

Some weights of the model checkpoint at ./LUKE-Marian-base-model were not used when initializing EncoderDecoderModel: ['encoder.encoder.layer.11.output.LayerNorm.weight', 'encoder.encoder.layer.11.attention.self.key.bias', 'encoder.encoder.layer.10.output.LayerNorm.weight', 'encoder.encoder.layer.6.attention.self.w2e_query.bias', 'encoder.encoder.layer.11.attention.output.LayerNorm.bias', 'encoder.encoder.layer.11.attention.output.dense.weight', 'encoder.encoder.layer.8.attention.output.LayerNorm.bias', 'encoder.encoder.layer.11.attention.self.query.weight', 'encoder.encoder.layer.8.intermediate.dense.bias', 'encoder.encoder.layer.11.attention.self.key.weight', 'encoder.encoder.layer.6.intermediate.dense.bias', 'encoder.encoder.layer.8.output.dense.bias', 'encoder.encoder.layer.7.attention.self.w2e_query.weight', 'encoder.encoder.layer.7.attention.output.LayerNorm.weight', 'encoder.encoder.layer.6.intermediate.dense.weight', 'encoder.encoder.layer.8.output.dense.weight', 'encoder.encod

EncoderDecoderModel(
  (encoder): LukeModel(
    (embeddings): LukeEmbeddings(
      (word_embeddings): Embedding(50267, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (entity_embeddings): LukeEntityEmbeddings(
      (entity_embeddings): Embedding(500000, 256, padding_idx=0)
      (entity_embedding_dense): Linear(in_features=256, out_features=768, bias=False)
      (position_embeddings): Embedding(514, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): LukeEncoder(
      (layer): ModuleList(
        (0): LukeLayer(
          (attention): LukeAttention(
            (self): LukeSelfAttention(
              (query): Linear(in_features=

In [ ]:
model.base_model.config.decoder_start_token_id

0

In [ ]:
model.base_model.config.pad_token_id

1

In [ ]:
tokenizer.pad_token_id

1

In [ ]:
model.config

EncoderDecoderConfig {
  "_commit_hash": null,
  "_name_or_path": "./LUKE-Marian-base-model",
  "architectures": [
    "EncoderDecoderModel"
  ],
  "bos_token_id": 0,
  "decoder": {
    "_name_or_path": "Helsinki-NLP/opus-mt-en-nl",
    "_num_labels": 3,
    "activation_dropout": 0.0,
    "activation_function": "swish",
    "add_bias_logits": false,
    "add_cross_attention": true,
    "add_final_layer_norm": false,
    "architectures": [
      "MarianMTModel"
    ],
    "attention_dropout": 0.0,
    "bad_words_ids": [
      [
        67027
      ]
    ],
    "bos_token_id": 0,
    "chunk_size_feed_forward": 0,
    "classif_dropout": 0.0,
    "classifier_dropout": 0.0,
    "cross_attention_hidden_size": null,
    "d_model": 512,
    "decoder_attention_heads": 8,
    "decoder_ffn_dim": 2048,
    "decoder_layerdrop": 0.0,
    "decoder_layers": 6,
    "decoder_start_token_id": 67027,
    "decoder_vocab_size": 67028,
    "diversity_penalty": 0.0,
    "do_sample": false,
    "dropout": 0.1,

# **Evaluator**

In [ ]:
%cd Utils

/content/drive/MyDrive/models/CodeGeneration/New/CoNaLa/LUKE-Marian/Utils


In [ ]:
from google.colab import files
import evaluator
evaluator = evaluator.CodeGenerationEvaluator(tokenizer, device,smooth_bleu=True)

/content/drive/MyDrive/models/CodeGeneration/New/CoNaLa/LUKE-Marian/Utils/evaluator.py:32: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  self.sacre_bleu: Metric = load_metric('sacrebleu')


In [ ]:
%cd ..

/content/drive/MyDrive/models/CodeGeneration/New/CoNaLa/LUKE-Marian




# **Prepare the data.**

In [ ]:
encoder_length = 32
decoder_length = 32
batch_size = 1


# map data correctly
def map_to_encoder_decoder_inputs(batch):
    inputs = tokenizer(batch["intent"], padding="max_length", truncation=True, max_length=encoder_length)
    outputs = tokenizer(batch["snippet"], padding="max_length", truncation=True, max_length=decoder_length)
    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["labels"] = outputs.input_ids.copy()
    batch["decoder_attention_mask"] = outputs.attention_mask

    """
    # complicated list comprehension here because pad_token_id alone is not good enough to know whether label should be excluded or not
    batch["labels"] = [
        [-100 if mask == 0 else token for mask, token in mask_and_tokens] for mask_and_tokens in [zip(masks, labels) for masks, labels in zip(batch["decoder_attention_mask"], batch["labels"])]
    ]
    """
    assert all([len(x) == encoder_length for x in inputs.input_ids])
    assert all([len(x) == decoder_length for x in outputs.input_ids])

    return batch

In [ ]:
# make train dataset ready
train_data = train_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=1, remove_columns=['intent', 'snippet'],
)
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)



  0%|          | 0/16000 [00:00<?, ?ba/s]

In [ ]:
# same for validation dataset
val_data = val_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=1, remove_columns=['intent', 'snippet'],
)
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)


  0%|          | 0/1000 [00:00<?, ?ba/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,
                                       max_length=512,padding=True, ####new
                                       model = model)

# **Training**

In [ ]:
# set training arguments - these params are not really tuned, feel free to change
training_args = Seq2SeqTrainingArguments(
    output_dir="./LUKE-marian-training",
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    predict_with_generate=True,
    num_train_epochs=10,
    do_train=True,
    do_eval=True,
    fp16=True,
    overwrite_output_dir=True,
    learning_rate = 1e-5,
    weight_decay=0.01,
    warmup_ratio = 0.05,
    seed = 1995,
    save_total_limit = 2,
    load_best_model_at_end = True,
    #push_to_hub=True,
)
#

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=evaluator,
    data_collator = data_collator,
    train_dataset=train_data,
    eval_dataset =val_data,

)
#    save_total_limit=3,
#

Using cuda_amp half precision backend


In [ ]:
# luke-marian-training
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 24687
  Num Epochs = 10
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 123440
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2279: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  "`max_length` is ignored when `padding`=`True` and there is no truncation strategy. "
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:530: FutureWarnin

Epoch,Training Loss,Validation Loss,Bleu,Sacrebleu,Bleu-unigram-precision,Bleu-bigram-precision,Bleu-trigram-precision,Rouge-2,Rouge-l,Sacre-unigram-precision,Sacre-bigram-precision,Sacre-trigram-precision
1,2.070800,2.000275,2.684491,1.999431,11.786486,4.437241,1.489482,6.205522,16.279011,9.165324,3.068266,1.122675
2,1.592900,1.566674,11.894925,10.517182,35.653685,16.504359,7.697832,9.285024,21.369709,32.189757,13.497039,6.933406
3,1.397600,1.348171,15.812048,14.502875,41.545279,20.380422,10.674213,11.507141,25.056115,37.705137,17.460467,10.044476
4,1.222500,1.210052,19.516018,18.045789,47.748469,25.784024,14.714507,14.779558,29.017327,44.762700,22.827850,14.272132
5,1.041900,1.113235,23.897739,22.391258,50.831576,29.933360,18.896720,17.659011,32.473741,48.197427,27.272727,18.576437
6,0.935900,1.030863,26.135634,24.691669,52.317543,31.787753,21.033597,19.285893,34.479392,49.452127,29.035703,20.320217
7,0.887100,0.978565,27.797343,26.347041,53.079502,33.024932,22.436396,21.424552,36.136642,50.590090,30.596181,22.048225
8,0.781300,0.943821,29.935432,28.497851,55.956442,35.982143,24.932622,23.420065,38.392035,53.447525,33.437983,24.481944
9,0.794600,0.917237,31.220446,29.946767,56.034163,36.765647,26.175869,24.827737,39.352766,53.705218,34.405358,25.914202
10,0.759200,0.909754,31.367998,30.104935,56.797668,37.434490,26.710288,25.442674,40.069889,54.426070,35.072194,26.401637


***** Running Evaluation *****
  Num examples = 1237
  Batch size = 1
INFO:seq_to_seq:Got BLEU of 2.68 and SacreBLEU of 2.00
Saving model checkpoint to ./luke-marian-training/checkpoint-12344
Configuration saved in ./luke-marian-training/checkpoint-12344/config.json
Model weights saved in ./luke-marian-training/checkpoint-12344/pytorch_model.bin
tokenizer config file saved in ./luke-marian-training/checkpoint-12344/tokenizer_config.json
Special tokens file saved in ./luke-marian-training/checkpoint-12344/special_tokens_map.json
added tokens file saved in ./luke-marian-training/checkpoint-12344/added_tokens.json
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2279: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  "`max_length` is ignored when `padding`=`True` and there is no truncation strategy. "
/usr/local/lib/python3.7/dist-packages/transformers/models/enc

TrainOutput(global_step=123440, training_loss=1.3350969454257888, metrics={'train_runtime': 12015.4288, 'train_samples_per_second': 20.546, 'train_steps_per_second': 10.273, 'total_flos': 3771355808563200.0, 'train_loss': 1.3350969454257888, 'epoch': 10.0})

In [ ]:
# luke-marian-training-1
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 24687
  Num Epochs = 10
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 123440
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2279: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  "`max_length` is ignored when `padding`=`True` and there is no truncation strategy. "
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:530: FutureWarnin

Epoch,Training Loss,Validation Loss,Bleu,Sacrebleu,Bleu-unigram-precision,Bleu-bigram-precision,Bleu-trigram-precision,Rouge-2,Rouge-l,Sacre-unigram-precision,Sacre-bigram-precision,Sacre-trigram-precision
1,0.774000,0.913110,32.081117,30.767355,56.603256,37.109205,26.487573,25.923226,40.443600,54.421252,34.863680,26.201225
2,0.653200,0.833840,34.145643,33.002799,61.275100,42.337954,32.169876,28.971835,43.230418,59.048754,40.190434,31.907054
3,0.611800,0.775908,39.591693,38.271005,61.689777,44.308745,34.669578,33.461963,46.656538,59.681660,42.369437,34.542202
4,0.532700,0.734530,41.583977,40.202755,64.702230,47.792102,38.445708,35.922338,48.890872,62.970691,46.017043,38.298595
5,0.436500,0.703773,43.539070,42.186144,66.379062,49.905732,40.825538,38.227821,51.363079,64.790067,48.254238,40.784983
6,0.395500,0.664523,46.185986,44.965694,67.892977,51.835734,42.847241,40.772889,53.474436,66.289284,50.163594,42.762793
7,0.380900,0.642006,48.530412,47.488511,67.829136,52.751822,44.542804,42.931361,54.935589,66.236365,51.321118,44.494843
8,0.334900,0.627401,49.571997,48.477357,70.682445,55.586973,47.143088,44.486592,56.711332,69.117142,53.981191,47.064738
9,0.347000,0.614759,50.427655,49.369013,69.827225,55.312315,47.118269,44.880113,56.820641,68.362587,53.783618,47.044375
10,0.335900,0.610225,50.871381,49.822873,69.567024,55.255299,47.159979,45.592518,57.143904,68.066667,53.736781,47.052791


***** Running Evaluation *****
  Num examples = 1237
  Batch size = 1
INFO:seq_to_seq:Got BLEU of 32.08 and SacreBLEU of 30.77
Saving model checkpoint to ./luke-marian-training-1/checkpoint-12344
Configuration saved in ./luke-marian-training-1/checkpoint-12344/config.json
Model weights saved in ./luke-marian-training-1/checkpoint-12344/pytorch_model.bin
tokenizer config file saved in ./luke-marian-training-1/checkpoint-12344/tokenizer_config.json
Special tokens file saved in ./luke-marian-training-1/checkpoint-12344/special_tokens_map.json
added tokens file saved in ./luke-marian-training-1/checkpoint-12344/added_tokens.json
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2279: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  "`max_length` is ignored when `padding`=`True` and there is no truncation strategy. "
/usr/local/lib/python3.7/dist-packages/transform

TrainOutput(global_step=123440, training_loss=0.4878676912032082, metrics={'train_runtime': 11854.8374, 'train_samples_per_second': 20.824, 'train_steps_per_second': 10.413, 'total_flos': 3771355808563200.0, 'train_loss': 0.4878676912032082, 'epoch': 10.0})

In [ ]:
trainer.save_model()
trainer.save_state()

Saving model checkpoint to ./LUKE-marian-training1
Configuration saved in ./LUKE-marian-training1/config.json
Model weights saved in ./LUKE-marian-training1/pytorch_model.bin
tokenizer config file saved in ./LUKE-marian-training1/tokenizer_config.json
Special tokens file saved in ./LUKE-marian-training1/special_tokens_map.json
added tokens file saved in ./LUKE-marian-training1/added_tokens.json


# **Visualize**

In [ ]:
%load_ext tensorboard

### **LUKE-marian**

In [ ]:
tensorboard --logdir ./LUKE-Marian-Training1/runs

<IPython.core.display.Javascript object>

# **Inference**

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
new_model = AutoModelForSeq2SeqLM.from_pretrained("./LUKE-marian-training1/")
new_model.to("cuda")

new_tokenizer = AutoTokenizer.from_pretrained("./LUKE-marian-training1/", use_fast=True)


In [ ]:
new_model

EncoderDecoderModel(
  (encoder): LukeModel(
    (embeddings): LukeEmbeddings(
      (word_embeddings): Embedding(50267, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (entity_embeddings): LukeEntityEmbeddings(
      (entity_embeddings): Embedding(500000, 256, padding_idx=0)
      (entity_embedding_dense): Linear(in_features=256, out_features=768, bias=False)
      (position_embeddings): Embedding(514, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): LukeEncoder(
      (layer): ModuleList(
        (0): LukeLayer(
          (attention): LukeAttention(
            (self): LukeSelfAttention(
              (query): Linear(in_features=

In [ ]:
new_model.config.max_length


512

# **Evaluator**

In [ ]:
%cd Utils

/content/drive/MyDrive/models/CodeGeneration/New/CoNaLa/LUKE-Marian/Utils


In [ ]:
from google.colab import files
import evaluator
evaluator = evaluator.CodeGenerationEvaluator(new_tokenizer, device,smooth_bleu=True)

In [ ]:
batch_size = 1

# map data correctly
def generate_new_code(batch):
    inputs = new_tokenizer(batch["intent"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")

    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = new_model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_str = new_tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred_code"] = output_str

    return batch


In [ ]:
results = test_dataset.map(generate_new_code, batched=True, batch_size=batch_size)

#results = test_dataset.map(generate_new_code, batched=True, batch_size=batch_size, remove_columns=["intent"])

  0%|          | 0/500 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1268: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [ ]:
%cd ..

/content/drive/MyDrive/models/CodeGeneration/New/CoNaLa/LUKE-Marian


In [ ]:
import re
import string

import datasets
import numpy as np

import evaluate
from evaluate import load

In [ ]:

exact_match = evaluate.load("exact_match")

#refs = ["The cat sat on the mat.", "Theaters are great.", "It's like comparing oranges and apples."]
#preds = ["The cat sat on the mat?", "Theaters are great.", "It's like comparing apples and oranges."]

"""
for i in range(len(results)):
  refs = results["code"][i]
  preds = results["pred_code"][i]
"""

'\nfor i in range(len(results)):\n  refs = results["code"][i]\n  preds = results["pred_code"][i]\n'

In [ ]:
refs = results["snippet"]
preds = results["pred_code"]

In [ ]:

exact_match_results = exact_match.compute(references=refs, predictions=preds, regexes_to_ignore=["▁", " "], ignore_case=True, ignore_punctuation=True, ignore_numbers=True)
print(round(exact_match_results["exact_match"], 5)*100)

7.6


In [ ]:
with open("./references.txt", 'w') as f:
  for i in range(len(results)):
    nl = results["intent"][i]
    ref = results["snippet"][i]
    pred = results["pred_code"][i]

    if pred is not None and pred != "":
      if ref is not None and ref != "":
        f.write(ref + '\n')
    else:
      continue


with open("./predictions.txt", 'w') as f:
  for i in range(len(results)):
    nl = results["intent"][i]
    ref = results["snippet"][i]
    pred = results["pred_code"][i]

    if pred is not None and pred != "":
      if ref is not None and ref != "":
        f.write(pred + '\n')
    else:
      continue


In [ ]:
# Ok Evaluator
bleu_score =0
rouge_score = 0
sacre_bleu = 0

for i in range(len(results)):
  ref = results["snippet"][i]
  pred = results["pred_code"][i]
  if pred is not None and pred != "":
      if ref is not None and ref != "":
        #bleu_metric = metric.compute(predictions=[pred], references=[[ref]])
        #bleu_score  += bleu_metric["score"]

        bleu_metric = evaluator.evaluate([pred], [ref])
        #bleu_metric = evaluator.evaluateSingle(pred, ref)

        #bleu_score += bleu_metric['BLEU-Unigram-Precision']
        bleu_score += bleu_metric['BLEU']
        rouge_score += bleu_metric['ROUGE-L']
        sacre_bleu += bleu_metric['SacreBLEU']
        #print(bleu_metric['SacreBLEU'])
  else:
    continue

# **Performance Metrics**

**BLEU Score**

In [ ]:
# luke-marian-training
print('Bleu Score: {} (scale 0-100)'.format(bleu_score/len(results)))

Bleu Score: 29.82811197677713 (scale 0-100)


**SacreBLEU**

In [ ]:
print('Sacre Bleu: {} (scale 0-100)'.format(sacre_bleu/len(results)))

Sacre Bleu: 25.31867897964577 (scale 0-100)


**Rouge Score**

In [ ]:
# luke-marian-training
print('ROUGE Score: {} (scale 0-100)'.format(rouge_score/len(results)))

ROUGE Score: 39.84307925421209 (scale 0-100)


**CodeBLEU**

In [ ]:
from google.colab import files

In [ ]:
import calc_code_bleu
ngram_match_score, weighted_ngram_match_score, syntax_match_score, dataflow_match_score, code_bleu_score = calc_code_bleu.calc_code_bleu("references.txt","predictions.txt","python", "keywords")

In [ ]:
print("ngram_match_score: ", ngram_match_score*100)
print("weighted_ngram_match_score: ", weighted_ngram_match_score*100)
print("syntax_match_score: ", syntax_match_score*100)
print("dataflow_match_score: ", dataflow_match_score*100)
print("code_bleu_score: ", code_bleu_score*100)

# **Save the  results**

In [ ]:
#%cd ..

/content/drive/MyDrive/models/CodeGeneration/MarianCG-CoNaLa-Large


In [ ]:
bleu_score = 0
with open("./Output.txt", 'w') as f:
  for i in range(len(results)):
    nl = results["intent"][i]
    ref = results["snippet"][i]
    pred = results["pred_code"][i]

    if pred is not None and pred != "":
      if ref is not None and ref != "":

        bleu_metric = evaluator.evaluate([pred], [ref])
        f.write( "NL: " + nl + '\n')
        f.write( "ref: " + ref + '\n')
        f.write( "Output: " + pred + '\n')lo
        f.write("bleu_metric: " + str(bleu_metric["BLEU"]) + '\n')
        f.write(10*"-" + '\n')
        bleu_score  += bleu_metric["BLEU"]

    else:
      continue


# **Test**

In [ ]:
# map data correctly
def generate_code(NL):
    inputs = new_tokenizer([NL], padding="max_length", truncation=True, max_length=512, return_tensors="pt")

    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = new_model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_code = new_tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return output_code

In [ ]:

#NL = "decode a hex string to utf-8."
NL = "remove first and last lines of string `s`"
output_code = generate_code(NL)

In [ ]:
output_code

In [ ]:
natural_language = "How I can get rid of None values in dictionary?"
gen_code = generate_code(natural_language)

In [ ]:
gen_code